# Medical Tables Cleaning

This notebook reloads both CSV exports, keeps every data column, and standardizes the headers so they are easy to read.

In [1]:
from pathlib import Path
import pandas as pd
import unicodedata

In [3]:
DATA_DIR = Path(".")

def _strip_header_value(value):
    if pd.isna(value):
        return None
    value = str(value).strip()
    if not value or value.lower().startswith('unnamed'):
        return None
    return value.replace('\n', ' ')

def _slug(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')
    text = text.lower().replace('-', ' ')
    text = ''.join(ch if ch.isalnum() or ch == ' ' else '_' for ch in text)
    return '_'.join(text.split())

def _build_equipment_columns(columns):
    frame = columns.to_frame(index=False)
    frame = frame.apply(lambda col: col.map(_strip_header_value))
    frame.iloc[:, 0] = frame.iloc[:, 0].ffill()
    names = []
    for first, second in frame.itertuples(index=False):
        parts = [part for part in (first, second) if part]
        if not parts:
            names.append(None)
        else:
            names.append(_slug(' '.join(parts)))
    if len(names) >= 3:
        names[0] = 'region'
        names[1] = 'region_note'
        names[2] = 'year'
    return names

def _clean_numeric(series):
    cleaned = (series.astype(str)
               .str.replace('\u00a0', '', regex=False)
               .str.replace(' ', '', regex=False)
               .str.replace(',', '.', regex=False)
               .replace({'-': None, '': None, 'nan': None}))
    return pd.to_numeric(cleaned, errors='coerce')

def clean_equipment(path):
    raw = pd.read_csv(path, header=[2, 3])
    raw.columns = _build_equipment_columns(raw.columns)
    raw = raw.loc[:, [c for c in raw.columns if c is not None]]
    raw = raw.dropna(axis=1, how='all')
    raw = raw.loc[:, ~raw.columns.duplicated()]
    if 'region_note' in raw.columns and raw['region_note'].isna().all():
        raw = raw.drop(columns='region_note')
    rename_map = {
        'aparat_kobaltowy_gammaknife_cobalt_apparatus_gammaknife_liczba_urzadzen_number_of_equipment': 'gamma_knife_devices',
        'aparat_kobaltowy_gammaknife_cobalt_apparatus_gammaknife_liczba_badan_number_of_diagnostic_exams': 'gamma_knife_exams',
        'medyczny_akcelerator_cyberknife_medical_accelerator_cyberknife_liczba_urzadzen_number_of_equipment': 'cyberknife_devices',
        'medyczny_akcelerator_cyberknife_medical_accelerator_cyberknife_liczba_badan_number_of_diagnostic_exams': 'cyberknife_exams',
        'respirator_ventilator_liczba_urzadzen_number_of_equipment': 'ventilator_devices',
        'respirator_ventilator_liczba_pacjentow_poddanych_terapii_number_of_patients_treated': 'ventilator_patients',
        'urzadzenie_do_pozaustrojowej_oksygenacji_krwi__ecmo__extracorporeal_membrane_oxygenation__ecmo__liczba_urzadzen_number_of_equipment': 'ecmo_devices',
        'urzadzenie_do_pozaustrojowej_oksygenacji_krwi__ecmo__extracorporeal_membrane_oxygenation__ecmo__liczba_pacjentow_poddanych_terapii_number_of_patients_treated': 'ecmo_patients',
        'urzadzenie_do_bezpiecznego_transportu_chorych_w_warunkach_penej_izolacji_oddechowej_device_for_safe_transport_of_patients_in_conditions_of_complete_respiratory_isolation_liczba_urzadzen_number_of_equipment': 'isolation_transport_devices',
        'urzadzenie_do_bezpiecznego_transportu_chorych_w_warunkach_penej_izolacji_oddechowej_device_for_safe_transport_of_patients_in_conditions_of_complete_respiratory_isolation_liczba_przetranspo_rtowanych_pacjentow_number_of_patients_transported': 'isolation_transport_patients'
    }
    raw = raw.rename(columns=rename_map)
    numeric_cols = [col for col in raw.columns if col not in {'region', 'year'}]
    for col in numeric_cols:
        raw[col] = _clean_numeric(raw[col])
    raw['year'] = _clean_numeric(raw['year'])
    raw = raw.dropna(subset=['region'])
    return raw.reset_index(drop=True)

def clean_hospitals(path):
    raw = pd.read_csv(path, skiprows=6)
    rename_map = {
        raw.columns[0]: 'region',
        raw.columns[2]: 'year',
        raw.columns[3]: 'hospital_count',
        raw.columns[4]: 'beds_total',
        raw.columns[5]: 'beds_children',
        raw.columns[6]: 'beds_per_10k',
        raw.columns[7]: 'patients_total',
        raw.columns[8]: 'patients_children',
        raw.columns[9]: 'patients_per_10k',
    }
    cleaned = raw.rename(columns=rename_map)[list(rename_map.values())]
    cleaned['region'] = cleaned['region'].astype(str).str.strip()
    numeric_cols = [col for col in cleaned.columns if col not in {'region'}]
    for col in numeric_cols:
        cleaned[col] = _clean_numeric(cleaned[col])
    cleaned = cleaned[cleaned['region'].notna() & (cleaned['region'].str.len() > 0)]
    return cleaned.reset_index(drop=True)

In [13]:
equipment_path = DATA_DIR / 'table1.csv'
equipment_clean = clean_equipment(equipment_path)[:18]
print(f'Equipment shape: {equipment_clean.shape}')
print('Equipment columns:', ', '.join(equipment_clean.columns))
equipment_clean.head()

Equipment shape: (18, 12)
Equipment columns: region, year, gamma_knife_devices, gamma_knife_exams, cyberknife_devices, cyberknife_exams, ventilator_devices, ventilator_patients, ecmo_devices, ecmo_patients, isolation_transport_devices, isolation_transport_patients


,region,year,gamma_knife_devices,gamma_knife_exams,cyberknife_devices,cyberknife_exams,ventilator_devices,ventilator_patients,ecmo_devices,ecmo_patients,isolation_transport_devices,isolation_transport_patients
0,POLSKA,2022.0,1.0,537.0,5.0,1045.0,17017.0,147779.0,172.0,813.0,2060.0,6966.0
1,POLAND,2023.0,2.0,1929.0,5.0,1563.0,17514.0,158379.0,188.0,875.0,810.0,4162.0
2,Dolnośląskie,NaN,NaN,NaN,NaN,NaN,1009.0,19456.0,11.0,49.0,198.0,489.0
3,Kujawsko-pomorskie,NaN,NaN,NaN,NaN,NaN,2008.0,5864.0,14.0,24.0,11.0,58.0
4,Lubelskie,NaN,NaN,NaN,NaN,NaN,818.0,14890.0,11.0,7.0,37.0,404.0


In [14]:
hospitals_path = DATA_DIR / 'table2.csv'
hospitals_clean = clean_hospitals(hospitals_path)[:18]
print(f'Hospital shape: {hospitals_clean.shape}')
print('Hospital columns:', ', '.join(hospitals_clean.columns))
hospitals_clean.head()

Hospital shape: (18, 9)
Hospital columns: region, year, hospital_count, beds_total, beds_children, beds_per_10k, patients_total, patients_children, patients_per_10k


,region,year,hospital_count,beds_total,beds_children,beds_per_10k,patients_total,patients_children,patients_per_10k
0,POLSKA,2022.0,896.0,164155.0,NaN,43.5,6895888.0,1191367.0,1823.0
1,POLAND,2023.0,894.0,161207.0,23500.0,42.8,7159490.0,1158499.0,1899.0
2,Dolnośląskie,NaN,78.0,13039.0,1719.0,45.3,544547.0,84204.0,1889.0
3,Kujawsko-pomorskie,NaN,43.0,8358.0,1402.0,41.9,351048.0,63947.0,1754.0
4,Lubelskie,NaN,45.0,9568.0,1059.0,47.6,373276.0,47463.0,1850.0


In [15]:
hospitals_clean

,region,year,hospital_count,beds_total,beds_children,beds_per_10k,patients_total,patients_children,patients_per_10k
0,POLSKA,2022.0,896.0,164155.0,NaN,43.5,6895888.0,1191367.0,1823.0
1,POLAND,2023.0,894.0,161207.0,23500.0,42.8,7159490.0,1158499.0,1899.0
2,Dolnośląskie,NaN,78.0,13039.0,1719.0,45.3,544547.0,84204.0,1889.0
3,Kujawsko-pomorskie,NaN,43.0,8358.0,1402.0,41.9,351048.0,63947.0,1754.0
4,Lubelskie,NaN,45.0,9568.0,1059.0,47.6,373276.0,47463.0,1850.0
5,Lubuskie,NaN,24.0,3766.0,484.0,38.6,155221.0,23788.0,1588.0
6,Łódzkie,NaN,62.0,11346.0,1655.0,48.0,513263.0,79378.0,2165.0
7,Małopolskie,NaN,69.0,13416.0,2104.0,39.1,595124.0,107457.0,1736.0
8,Mazowieckie,NaN,116.0,23214.0,3707.0,42.1,1091923.0,203401.0,1982.0
9,Opolskie,NaN,29.0,4030.0,384.0,43.0,154586.0,15856.0,1645.0


In [16]:
# Save hospitals_clean to a CSV file
hospitals_clean.to_csv(DATA_DIR / 'hospitals_clean.csv', index=False, encoding='utf-8-sig')
print("✓ hospitals_clean saved to 'hospitals_clean.csv'")

# Save equipment_clean to a CSV file
equipment_clean.to_csv(DATA_DIR / 'equipment_clean.csv', index=False, encoding='utf-8-sig')
print("✓ equipment_clean saved to 'equipment_clean.csv'")

✓ hospitals_clean saved to 'hospitals_clean.csv'
✓ equipment_clean saved to 'equipment_clean.csv'


In [17]:
equipment_clean

,region,year,gamma_knife_devices,gamma_knife_exams,cyberknife_devices,cyberknife_exams,ventilator_devices,ventilator_patients,ecmo_devices,ecmo_patients,isolation_transport_devices,isolation_transport_patients
0,POLSKA,2022.0,1.0,537.0,5.0,1045.0,17017.0,147779.0,172.0,813.0,2060.0,6966.0
1,POLAND,2023.0,2.0,1929.0,5.0,1563.0,17514.0,158379.0,188.0,875.0,810.0,4162.0
2,Dolnośląskie,NaN,NaN,NaN,NaN,NaN,1009.0,19456.0,11.0,49.0,198.0,489.0
3,Kujawsko-pomorskie,NaN,NaN,NaN,NaN,NaN,2008.0,5864.0,14.0,24.0,11.0,58.0
4,Lubelskie,NaN,NaN,NaN,NaN,NaN,818.0,14890.0,11.0,7.0,37.0,404.0
5,Lubuskie,NaN,NaN,NaN,NaN,NaN,721.0,3123.0,5.0,6.0,2.0,7.0
6,Łódzkie,NaN,NaN,NaN,NaN,NaN,1093.0,7735.0,15.0,52.0,214.0,100.0
7,Małopolskie,NaN,NaN,NaN,1.0,369.0,1627.0,14023.0,17.0,312.0,36.0,156.0
8,Mazowieckie,NaN,1.0,1389.0,1.0,200.0,2539.0,20525.0,38.0,191.0,92.0,1551.0
9,Opolskie,NaN,NaN,NaN,NaN,NaN,441.0,2380.0,4.0,18.0,36.0,40.0
